In [1]:
import pandas as pd
import ast 

In [2]:
# Tomamos la ruta relativa de los archivos
ruta_movies = (r'../data_default/movies_dataset.csv')
ruta_credits = (r'../data_default/credits.csv')



In [3]:
#Se crea una funcion para desanidar y automatizar el proceso
def desanidar_columna(df, columna, claves):
    # Convierte con "ast.literal_eval" el contenido de la columna de una cadena en un objeto Python (lista o diccionario)
    df[columna] = df[columna].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)
    
    # Verifica si el primer elemento no nulo de la columna es una lista
    if isinstance(df[columna].dropna().iloc[0], list):
        # Si es una lista, para cada clave crea una nueva columna combinando los valores de la lista
        for clave in claves:
            df[f'{columna}_{clave}'] = df[columna].apply(lambda x: ', '.join([str(item.get(clave, '')) for item in x]) if isinstance(x, list) else None)
    
    # Verifica si el primer elemento no nulo de la columna es un diccionario
    elif isinstance(df[columna].dropna().iloc[0], dict):
        # Si es un diccionario, para cada clave en claves, crea una nueva columna con el valor correspondiente de la clave
        for clave in claves:
            df[f'{columna}_{clave}'] = df[columna].apply(lambda x: x.get(clave) if isinstance(x, dict) else None)
    
    # Elimina la columna original después de desanidar
    df = df.drop(columns=[columna])
    
    # Devuelve el DataFrame modificado
    return df


In [10]:
#Cargamos los datos (Leemos solo las columnas que necesitamos del archivo)
columnas_deseadas = ['id', 'popularity', 'budget', 'release_date', 'revenue','runtime', 'title', 'vote_average', 'vote_count',
        'belongs_to_collection', 'production_companies', 'production_countries', 'genres']
movies_df = pd.read_csv(ruta_movies, usecols=columnas_deseadas, dtype={'column_name': str})

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3336\3682453663.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(ruta_movies, usecols=columnas_deseadas, dtype={'column_name': str})


In [11]:
#Cambiamos el formato de la columna a tipo date
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

#Eliminamos las fechas anteriores a el año 1975 (para optimizar memoria y a la vez trabajar con datos relevantes)
fecha_limite = pd.to_datetime('1975-01-01')
#Conservamos los id de las peliculas que vamos a eliminar para luego eliminarlas tmb de credits
ids_a_eliminar = movies_df[movies_df['release_date'] < fecha_limite]['id']
# Filtrar el DataFrame para conservar solo las filas con 'release_date' a partir de la fecha límite
movies_df = movies_df[movies_df['release_date'] >= fecha_limite]

In [12]:
#Desanidamos columnas de movies con la funcion creada anteriormente
movies_df = desanidar_columna(movies_df, 'belongs_to_collection', ['name'])
movies_df = desanidar_columna(movies_df, 'production_companies', ['name'])
movies_df = desanidar_columna(movies_df, 'production_countries', ['name'])
movies_df = desanidar_columna(movies_df, 'genres', ['name'])

In [13]:
# -Procesamos movies- 
#Eliminados los duplicados (id o titulo)
movies_df = movies_df.drop_duplicates(subset=['id', 'title'])
#Rellenamos los valores nulos con 0
movies_df['revenue'] = movies_df['revenue'].fillna(0)
movies_df['budget'] = movies_df['budget'].fillna(0)
#Creamos la columna year y eliminamos los que tienen valores nulos
movies_df['release_year'] = movies_df['release_date'].dt.year
movies_df = movies_df.dropna(subset=['release_date'])
# Eliminar cualquier fila donde 'release_date' no es una fecha válida
movies_df = movies_df[pd.to_datetime(movies_df['release_date'], errors='coerce').notna()]
# Cambiamos a dato numerico
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce').fillna(0)
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce').fillna(0)
# Calculamos el retorno de inversión para cada película en el DataFrame
movies_df['return'] = movies_df.apply(
    lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [14]:
# -Procesamos credits-
#Leemos el archivo
credits_df = pd.read_csv(ruta_credits, dtype={'column_name': str})
#Eliminamos los datos de las peliculas anteriores a 1975
credits_df[~credits_df['id'].isin(ids_a_eliminar)]
#Desanidamos columnas con la funcion creada
credits_df = desanidar_columna(credits_df, 'cast', ['name'])
credits_df = desanidar_columna(credits_df, 'crew', ['name', 'job'])
#Combinamos las columnas en formato listas en pares de (nombre,rol) y devuelve los nombres asociados a "director"
credits_df['director'] = credits_df.apply(
    lambda row: next((nombre for nombre, rol in zip(row['crew_name'].split(', '), row['crew_job'].split(', ')) if rol == 'Director'), None), axis=1)
#Borramos las columnas utilizadas
credits_df = credits_df.drop(columns=['crew_name', 'crew_job'])
#Recortamos a 2 elementos la columna cast name
credits_df['cast_name'] = credits_df['cast_name'].apply(
    lambda x: ', '.join(x.split(', ')[:2]) if pd.notnull(x) else None)

In [15]:
# *** Uno los Dataframes por su id para trabajar desde un solo df ***
# Transformo el "id" a numerico en ambos dataframes
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
credits_df['id'] = pd.to_numeric(credits_df['id'], errors='coerce')
# Elimino los Nulos
movies_df = movies_df.dropna(subset=['id'])  # Elimina filas con NaN en 'id'
credits_df = credits_df.dropna(subset=['id'])
#Ahora si uno los df por el id
data = pd.merge(movies_df, credits_df, on='id', how='left')

In [16]:
data.columns

Index(['budget', 'id', 'popularity', 'release_date', 'revenue', 'runtime',
       'title', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'production_companies_name', 'production_countries_name', 'genres_name',
       'release_year', 'return', 'cast_name', 'director'],
      dtype='object')

In [17]:
#Transformo los datos a numerico para que no me de error luego
data['popularity'] = pd.to_numeric(data['popularity'], errors='coerce')
data['budget'] = pd.to_numeric(data['budget'], errors='coerce')
data['revenue'] = pd.to_numeric(data['revenue'], errors='coerce')
data['vote_count'] = pd.to_numeric(data['vote_count'], errors='coerce')
data['vote_average'] = pd.to_numeric(data['vote_average'], errors='coerce')
data['runtime'] = pd.to_numeric(data['runtime'], errors='coerce')
data['return'] = pd.to_numeric(data['return'], errors='coerce')

In [18]:
#Guardo el df general para analizarlo
data.to_csv(r'../data_default/data.csv', index=False)

In [19]:
#Guardo el df en diferentes archivos .parquet para optimizar el uso de memoria
#Archivo con datos numericos
num_columnas = ['id', 'return', 'budget', 'revenue','vote_count', 'vote_average',  'popularity', 'release_date', 'release_year']
data[num_columnas].to_parquet(r'../data/data_num.parquet')
#Archivo con informacion sobre la pelicula
info_columnas = ['id', 'title','belongs_to_collection_name', 'genres_name', 'runtime']
data[info_columnas].to_parquet(r'../data/data_info.parquet')
#Archivo con informacion sobre la produccion de la pelicula
prod_columnas = ['id','director', 'production_companies_name', 'production_countries_name']
data[prod_columnas].to_parquet(r'../data/data_prod.parquet')
#Archivo con informacion sobre el reparto
cast_columnas = ['id', 'cast_name']
data[cast_columnas].to_parquet(r'../data/data_cast.parquet')
